<a href="https://colab.research.google.com/github/41371125h-chinrouzhen/114-1-PL/blob/main/HW2_%E6%88%90%E7%B8%BE%E4%B8%80%E6%9C%AC%E9%80%9A%EF%BC%88AI%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **成績一本通**
目標：從 Sheet 讀各次作業/測驗 → 計平時/期末/總成績 → 回寫成績總表。

AI 點子：針對「開放式作答」欄位，用模型產出整體摘要與常見迷思整理（不評分，只做總結）。

Google sheet鏈接:https://docs.google.com/spreadsheets/d/1HA1cvniLmO1O362GtHlrLEK3TQ8_XQJBULMpbaU-29s/edit?usp=sharing

注記：API key不可用，無法生成相關總結


In [1]:
# 套件與認證
import pandas as pd
import numpy as np
import gspread
from google.colab import auth
from google.auth import default
import time
from datetime import datetime
import getpass

auth.authenticate_user()
creds, _ = default()
client = gspread.authorize(creds)

# Google Sheet
SHEET_URL = "https://docs.google.com/spreadsheets/d/1HA1cvniLmO1O362GtHlrLEK3TQ8_XQJBULMpbaU-29s/edit?usp=sharing"
RECORDS_SHEET = "Records"
SUMMARY_SHEET = "Summary"

# Reecord
def read_records_df():
    ws = client.open_by_url(SHEET_URL).worksheet(RECORDS_SHEET)
    data = ws.get_all_values()
    if not data:
        return pd.DataFrame()
    df = pd.DataFrame(data[1:], columns=data[0])
    numeric_cols = ["作業1","作業2","作業3","小測1","小測2","期末"]
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)
    if '開放式作答' in df.columns:
        df['開放式作答'] = df['開放式作答'].fillna("").astype(str)
    else:
        df['開放式作答'] = ""
    return df

df = read_records_df()

# Cal
homework_cols = ["作業1","作業2","作業3"]
quiz_cols = ["小測1","小測2"]
df['平時成績'] = df[homework_cols + quiz_cols].mean(axis=1)
df['總成績'] = df['平時成績']*0.4 + df['期末']*0.6


# 寫回
try:
    summary_ws = client.open_by_url(SHEET_URL).worksheet(SUMMARY_SHEET)
    summary_ws.clear()
except gspread.WorksheetNotFound:
    summary_ws = client.open_by_url(SHEET_URL).add_worksheet(title=SUMMARY_SHEET, rows="100", cols="20")

summary_ws.update([df.columns.values.tolist()] + df.values.tolist())
print(f"已更新")



已更新


In [3]:
# 使用 Gemini API 生成摘要
import google.generativeai as genai
import getpass

try:
    # 手動輸入 API Key
    GOOGLE_API_KEY = getpass.getpass("請輸入你的 Gemini API Key: ")
    genai.configure(api_key=GOOGLE_API_KEY)

    # 選用 Gemini 模型
    gemini_model = genai.GenerativeModel("gemini-1.5-flash")

    # 合併所有開放式作答
    all_responses = "\n".join(df['開放式作答'].dropna().tolist())

    # Prompt
    prompt = f"""請根據以下學生的開放式作答內容，產出整體摘要與常見迷思整理：

{all_responses}

整體摘要：
常見迷思：
"""

    # 生成回答
    response = gemini_model.generate_content(prompt)

    print("===== 整體摘要與常見迷思整理 =====")
    print(response.text)

except Exception as e:
    print(f"❌ 無法使用 AI 模型生成摘要，錯誤訊息：{e}")


KeyboardInterrupt: Interrupted by user